In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Download stock data from Yahoo Finance
def download_stock_data(ticker, period='5y'):
    stock_data = yf.download(ticker, period=period)
    return stock_data


import requests


# Step 2: Preprocess the data
def preprocess_data(data, feature_col='Close', seq_length=60):
    # Use 'Close' prices to predict trends
    data = data[[feature_col]]

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences of data points for LSTM input
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)

    # Reshape the data to be compatible with LSTM (samples, timesteps, features)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler

# Step 3: Build the LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))  # Predicting a single output value (next price)

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the LSTM model
def train_lstm_model(model, X_train, y_train, epochs=50, batch_size=64):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

# Step 5: Make predictions and evaluate
def predict_and_evaluate(model, X_test, y_test, stock_scaler):
    predictions = model.predict(X_test)

    # Inverse transform only the stock price column
    predictions_stock_price = stock_scaler.inverse_transform(predictions)
    y_test_stock_price = stock_scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate the Root Mean Squared Error (RMSE) on the stock prices
    rmse = np.sqrt(np.mean((predictions_stock_price - y_test_stock_price) ** 2))
    return predictions_stock_price, rmse


In [2]:
# Step 6: Trading simulation logic
def simulate_trading(predictions, actual_prices, dates, initial_balance=10000, shares=0):
    balance = initial_balance
    total_shares = shares
    trade_log = []

    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        actual_price = actual_prices[i]
        date = dates[i]  # Make sure 'date' is a datetime object

        # Ensure you're not comparing dates with prices
        if predicted_price > actual_prices[i-1] and balance > actual_price:
            shares_to_buy = balance // actual_price
            balance -= shares_to_buy * actual_price
            total_shares += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_price} on {date}, Balance: {balance}, Shares: {total_shares}")

        elif predicted_price < actual_prices[i-1] and total_shares > 0:
            balance += total_shares * actual_price
            trade_log.append(f"Sold {total_shares} shares at {actual_price} on {date}, Balance: {balance}")
            total_shares = 0

    # Final balance after selling any remaining shares
    if total_shares > 0:
        balance += total_shares * actual_prices[-1]
        trade_log.append(f"Final Sale of {total_shares} shares at {actual_prices[-1]} on {dates[-1]}, Final Balance: {balance}")

    profit_loss = balance - initial_balance
    return trade_log, profit_loss

# Step 7: Run the entire pipeline with trading simulation
def run_stock_prediction_with_simulation(ticker, period='5y', seq_length=60):
    # Download and preprocess the data
    stock_data = download_stock_data(ticker, period)
    X, y, scaler = preprocess_data(stock_data, seq_length=seq_length)

    # Get the corresponding dates for the test set
    dates = stock_data.index[seq_length:]  # Dates aligned with the sequences

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Adjust the corresponding dates for the test set
    test_dates = dates[-len(X_test):]

    # Create and train the LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], 1))
    model = train_lstm_model(model, X_train, y_train, epochs=10)

    # Make predictions and evaluate the model
    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)

    # Rescale the actual test prices
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Simulate trading based on model predictions and include dates
    trade_log, profit_loss = simulate_trading(predictions.flatten(), actual_prices.flatten(), test_dates)

    print(f"Root Mean Squared Error: {rmse}")
    print(f"Final Profit/Loss: {profit_loss}")
    for log in trade_log:
        print(log)

    return stock_data, predictions, trade_log, profit_loss

# Example usage:
if __name__ == "__main__":
    ticker_symbol = 'GOOG '
    stock_data, predictions, trade_log, profit_loss = run_stock_prediction_with_simulation(ticker_symbol)


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - loss: 0.1010
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0080
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0062
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0040
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0031
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0028
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0028
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0023
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - loss: 0.0024
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0023
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
Root Mean Squared Error: 5.471712813333702
Final Profit/Loss: 498.1988525390607
Bought 74.0 shares at 133.91999816894534 on 2023-11-30 00:00:00+00:00, Balance: 89.92013549804506, Shares: 74.0
Sold 74.0 shares at 136.63999938964847 on 2023-12-08 00:00:00+00:00, Balance: 10201.280090332031
Bought 76.0